In [7]:
import sqlite3
from sqlite3 import Error

In [8]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
        
    except Error as e:
        print(e)
    
    #finally:
        #if conn:
            #conn.close()



db = create_connection(r"database/wallet_db.db")

In [9]:
def execute_w_res(conn, query):
    try:
        c = conn.cursor()
        result = c.execute(query)
        return result.fetchall()
    except Error as e:
        print(e)
        return None

def execute(conn, query):
    try:
        c = conn.cursor()
        c.execute(query)
        return True
    except Error as e:
        print(e)
        return None

In [66]:
def create_wallet_table(conn):
    query1 =  f"CREATE TABLE IF NOT EXISTS Wallets ( xprv text NOT NULL PRIMARY KEY,\n "
    query2 = f"name text UNIQUE, words text NOT NULL) WITHOUT ROWID;"
    query = query1+query2
    #print(query)
    return execute(conn, query)

def create_address_table(conn):
    query1 =f"CREATE TABLE IF NOT EXISTS Addresses ( address text NOT NULL PRIMARY KEY,\nacc_index INT NOT NULL,"
    query2 = "\npath text NOT NULL,\nchange_addr INT NOT NULL,\ncreated INT NOT NULL,\nwallet text NOT NULL,\nFOREIGN KEY (wallet) "
    query3 = "\nREFERENCES Wallets(xprv) ) WITHOUT ROWID ;"
    query = query1 + query2 + query3
    #print(query)
    return execute(conn, query)

def create_utxo_table(conn):
    query1 =f"CREATE TABLE IF NOT EXISTS Utxos ( address text NOT NULL,\namount INT NOT NULL,\ntx_id text NOT NULL,"
    query2 = "\nout_index INT NOT NULL,\ncreated INT NOT NULL,\nspent INT NOT NULL,\nconfirmed INT NOT NULL, "
    query3 = "\nPRIMARY KEY (tx_id, out_index)\nFOREIGN KEY (address)\nREFERENCES Addresses(address) );"
    query = query1 + query2 + query3
    #print(query)
    return execute(conn, query)

def create_transaction_table(conn):
    query1 =f"CREATE TABLE IF NOT EXISTS Transactions ( tx_id text NOT NULL PRIMARY KEY,\nlock_time INT,\nversion INT,\n"
    query2 = "\nn_confirmations INT NOT NULL,\ncreated INT NOT NULL)  WITHOUT ROWID ;"
    query = query1 + query2
    #print(query)
    return execute(conn, query)

def create_tx_in_table(conn):
    query1 =f"CREATE TABLE IF NOT EXISTS Tx_Ins ( tx_id text NOT NULL, out_index INT NOT NULL,\nspent_by text NOT NULL,\n"
    query2 = "FOREIGN KEY (tx_id,out_index)\nREFERENCES Utxo(tx_id,out_index) \n "
    query3 = "FOREIGN KEY (spent_by)\nREFERENCES Transactions(tx_id) )  WITHOUT ROWID ;"
    query = query1 + query2 + query3
    #print(query)
    return execute(conn, query)

def create_tx_out_table(conn):
    query1 =f"CREATE TABLE IF NOT EXISTS Tx_Outs ( out_index INT NOT NULL,\n amount INT NOT NULL,\ncreated_by text NOT NULL,\n"
    query2 = "script_pubkey text NOT NULL, \n PRIMARY KEY (created_by, out_index)\n"
    query3 = "FOREIGN KEY (created_by)\nREFERENCES Transactions(tx_id) )  WITHOUT ROWID ;"
    query = query1 + query2 + query3
    #print(query)
    return execute(conn, query)

def create_tables(conn):
    create_wallet_table(conn)
    create_address_table(conn)
    create_utxo_table(conn)
    create_transaction_table(conn)
    create_tx_in_table(conn)
    create_tx_out_table(conn)
    return True


def new_wallet(conn, xprv, words, name = None ):
    """
    Creates a new wallet in the database.
    conn: conn: internal database driver transaction.
    xprv: String; extended private key of the wallet.
    words: String; The mnemonic phrase of the wallet.
    name: String Optional. This is an alias for the wallet if user prefers it.
    """
    query1 = "INSERT INTO Wallets (xprv, words, name)\n "
    query2 = f"VALUES('{xprv}', '{words}', '{name}') ;"
    query = query1+query2
    #print(query)
    return execute(conn, query)

def new_address(conn, address, path, acc_index, change_addr, created, wallet):
    """
    Creates a new address in the database.
    conn: internal database driver transaction.
    address: String; the address to create.
    path: String; the path in wich the address is created until the account level. i.e. "m/44'/0'/0'/"
    as a standard, it must start with m and end with /
    acc_index: int; account index. This would be the last part of the path.
    change_addr: int, 0=False, 1=True; if the address is a "change adrress" then 1 (True). Otherwise 0 (False).
    created: int; timestamp.
    wallet: String; the xtended private key of the wallet.
    """
    query1 = "INSERT INTO Addresses (address, path, acc_index, change_addr, created, wallet)\n "
    query2 = f'VALUES("{address}", "{path}", {acc_index}, {change_addr}, {created}, "{wallet}") ;'
    query = query1+query2
    #print(query)
    return execute(conn, query)

def new_utxo(conn, address, amount, tx_id, out_index, created, spent=0, confirmed=0):
    """
    Creates new utxo in the database.
    conn: internal database driver transaction.
    address: the base58 representation of the address that holds the utxo.
    tx_id: String, the tx id or hash of the transaction the generated the utxo.
    out_index: Int, the index of the output in the transaction that generated the utxo.
    amount: Int, the amount of SATOSHIS that the utxo holds.
    created: timestamp
    spent: Int, 0=False, 1=True; if the utxo has been spent by a transaction then 1 (True).
    confirmed: Int, 0=False, 1=True; if the transaction has less than 6 confirmations then 0 (False).
    """
    query1 = "INSERT INTO Utxos (address, amount, tx_id, out_index, created, spent, confirmed)\n "
    query2 = f"VALUES('{address}', {amount}, '{tx_id}', {out_index}, {created}, {spent}, {confirmed});"
    query = query1+query2
    #print(query)
    return execute(conn, query)

def new_tx(conn, tx_id, tx_ins, tx_outs, created n_confirmations = 0, lock_time=0, version=1 ):
    """
    tx_id: String. transaction id.
    tx_ins: List of touples: [ (prev_tx_id, index), ... ]
    tx_outs: List of touples: [ (out_index, amount, script_pubkey), ... ]
    created: int; timestamp.
    n_confirmations: int; number of confirmations in the blockchain.
    lock_time: Int: transaction locktime.
    version: Int: version.
    """
    query1 = "INSERT INTO Transactions ( tx_id, created, n_confirmations, lock_time, version)\n "
    query2 = f"VALUES('{tx_id}', {created}, {n_confirmations}, {lock_time}, {version});"
    query = query1+query2
    execute(conn, query)
    
    for tx_in in tx_ins:
        query3 = "INSERT INTO Tx_Ins ( tx_id, out_index, spent_by)\n "
        query4 = f"VALUES( '{tx_in[0]}', {tx_in[1]}, '{tx_id}');"
        query = query3+query4
        execute(conn, query)
        
    for tx_out in tx_outs:
        query5 = "INSERT INTO Tx_Outs ( out_index, amount, script_pubkey, created_by)\n "
        query6 = f"VALUES( {tx_out[0]}, {tx_out[1]}, '{tx_out[2]}', '{tx_id}');"
        query = query5+query6
        execute(conn, query)
    
    return True


def update_confirmations(conn, tx_id, n_confirmations):
    """
    Updates the number of confirmations for a transaction to later confirm the transaction was broadcasted.
    conn: internal database driver transaction.
    tx_id: String, the id or hash of the transaction being broadcasted.
    n_confirmations: Int, the number of confirmations.
    """
    query = f"UPDATE Transactions \nSET n_confirmations = {n_confirmations} WHERE tx_id = '{tx_id}'"
    return execute(conn, query)
        

def spend_utxo(conn, tx_id, out_index):
    query = f"UPDATE Utxos \nSET spent = 1 \nWHERE tx_id = '{tx_id}' AND out_index = {out_index} "
    return execute(conn, query)
    
def spend_utxo(conn, tx_id, out_index):
    """
    Updates the state of an exisiting utxo to "spent=True" once the transaction was broadcasted successfully and confirmed.
    conn: internal database driver transaction.
    tx_id: String, the id or hash of the transaction that created the utxo (previous transaction).
    out_index: index of the utxo in the previous transaction.
    """
    query = f"UPDATE Utxos \nSET spent = 1 \nWHERE tx_id = '{tx_id}' AND out_index = {out_index} "
    return execute(conn, query)

def clean_addresses(conn):
    """
    Not necessary in this implementation.
    """
    pass

def look_for_coins(conn, wallet):
    """
    Searches the database for utxos that haven't been spent for the specific wallet.
    conn: internal database driver transaction.
    wallet: String, The wallet extended private key.
    Returns: List of touples [(tx_id, out_index, amount)]
    """
    query1 = f"SELECT tx_id, out_index, amount\n FROM Utxos INNER JOIN Addresses \n"
    query2 = f"ON Utxos.address = Addresses.address\nWHERE Utxos.spent = 0 AND Addresses.wallet = '{wallet}';"
    query = query1 + query2
    return execute_w_res(conn, query)
    
def get_unused_addresses(conn,wallet, days_range=None, max_days=None):
    """
    Searches the database for addresses that haven't been used in the specific wallet.
    tx: internal database driver transaction.
    xprv: String, The wallet extended private key.
    max_days: how far away should the app look for the unused address in the pass. The limit is 30 since the app deletes 
    unused addresses after a month of creation.
    days_range: number of days for the range in which to look for the address. The range will start
    in day_range days before the max_days day (starting_day = (max_days - days_range)).

    days_range and max_days should be either both specified or neither specified. If they are not, both values will be
    automatically set to 30 making the search to happen from now to the last 30 days. If only one of them is specified, 
    the function will throw an exception.
    """
    if days_range is None and max_days is None:
        query1 = f"SELECT address \nFROM Addresses WHERE\nNOT EXISTS(\nSELECT 1 \n FROM Utxos"
        query2 = f"\nWHERE Utxos.address = Addresses.address) \nAND\n Addresses.wallet = '{wallet}';"
        query = query1 + query2
        #print(query)
        return execute_w_res(conn, query)
    
    elif (days_range is None or max_days is None) and (days_range != max_days):
        raise Exception("Arguments days_range and max_days must be either both specified or neither one specified.")
            
    else:
        MILSEC_PER_DAY = 86400000
        finish_day = max_days * MILSEC_PER_DAY
        start_day = finish_day - days_range*MILSEC_PER_DAY
        now = 0#SOLVE THIS!!!!
        query1 = f"SELECT address \nFROM Addresses WHERE\nNOT EXISTS(\nSELECT 1 \n FROM Utxos"
        query2 = f"\nWHERE Utxos.address = Addresses.address) \nAND\n Addresses.wallet = '{wallet}'"
        query3 = f"\nAND\n ({now} - Addresses.created) > {start_day} AND ({now} - Addresses.created) < {finish_day} ';"
        query = query1 + query2 + query3
        #print(query)
        return execute_w_res(conn, query)
    
def get_all_addresses(conn,wallet):
    """
    Returns all the addresses in the database for the specific wallet.
    conn: internal database driver transaction.
    wallet: String, The wallet extended private key.
    """
    query = f"SELECT address \nFROM Addresses WHERE wallet = '{wallet}'; "
    return execute_w_res(conn, query)

def exist_utxo(conn, tx_id, out_index, confirmed):
    """
    Returns True if the utxo already exists in the database. False if it doesn't. It also updates
    the "confirmed" property of the utxo if necessary.
    tx: internal database driver transaction.
    tx_id: String, the tx id or hash of the transaction the generated the utxo.
    out_index: Int, the index of the output in the transaction that generated the utxo.
    confirmed: Int/Boolean 0=False, 1=True; the current state of confirmation of the transaction in the actual blockchain.
    """
    query = f"SELECT * FROM Utxos WHERE  tx_id = '{tx_id}' AND out_index = {out_index}"
    result = execute_w_res(conn, query)
    if len(result)>0:
        if result[0][6] != confirmed:
            query = f"UPDATE Utxos \nSET confirmed = {confirmed} WHERE tx_id = '{tx_id}' AND out_index = {out_index} "
            result = execute_w_res(conn, query)
            
    return True

def does_table_exist(conn, table):
    query = f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table}';"
    return execute_w_res(conn, query)

#def create_address(conn, address,i,change_addr,wallet_xprv):
    

In [40]:
res = create_tables(db)
res

CREATE TABLE IF NOT EXISTS Addresses ( address text NOT NULL PRIMARY KEY,
acc_index INT NOT NULL,
path text NOT NULL,
change_addr INT NOT NULL,
created INT NOT NULL,
wallet text NOT NULL,
FOREIGN KEY (wallet) 
REFERENCES Wallets(xprv) ) WITHOUT ROWID ;
CREATE TABLE IF NOT EXISTS Utxos ( address text NOT NULL,
amount INT NOT NULL,
tx_id text NOT NULL,
out_index INT NOT NULL,
created INT NOT NULL,
spent INT NOT NULL,
confirmed INT NOT NULL, 
PRIMARY KEY (tx_id, out_index)
FOREIGN KEY (address)
REFERENCES Addresses(address) );
CREATE TABLE IF NOT EXISTS Transactions ( tx_id text NOT NULL PRIMARY KEY,
lock_time INT,
version INT,

confirmations INT NOT NULL,
created INT NOT NULL)  WITHOUT ROWID ;
CREATE TABLE IF NOT EXISTS Tx_Ins ( tx_id text NOT NULL, out_index INT NOT NULL,
spent_by text NOT NULL,
FOREIGN KEY (tx_id,out_index)
REFERENCES Utxo(tx_id,out_index) 
 FOREIGN KEY (spent_by)
REFERENCES Transactions(tx_id) )  WITHOUT ROWID ;
PRIMARY KEY missing on table Tx_Ins
CREATE TABLE IF NOT 

True

In [17]:
res = create_address_table(db)

CREATE TABLE IF NOT EXISTS Addresses ( address text PRIMARY KEY,
acc_index INT NOT NULL UNIQUE,
change_addr INT NOT NULL,
created INT NOT NULL,
wallet text NOT NULL,
FOREIGN KEY (wallet) 
REFERENCES Wallets(xprv) ) WITHOUT ROWID ;


In [18]:
res

True

In [29]:
res = does_table_exist(db,"Wallets")

In [30]:
res

[]

In [25]:
f = db.cursor().execute("DROP TABLE Addresses")

In [6]:
db.close()

In [43]:
xprv ="tprv8ZgxMBicQKsPfQJYjuFAso9x6STzmUdMh5U8CQqqQUTgtQHBHCq4C7FseeeZg15L16UeSwbrLwJRTXNPQsJQwqvbBA11sn4M6c3jR1LwAQP"
words = "engine over neglect science fatigue dawn axis parent mind man escape era goose border invest slab relax bind desert hurry useless lonely frozen morning"
res = new_wallet(db, xprv, words)

INSERT INTO Wallets (xprv, words, name)
 VALUES('tprv8ZgxMBicQKsPfQJYjuFAso9x6STzmUdMh5U8CQqqQUTgtQHBHCq4C7FseeeZg15L16UeSwbrLwJRTXNPQsJQwqvbBA11sn4M6c3jR1LwAQP', 'engine over neglect science fatigue dawn axis parent mind man escape era goose border invest slab relax bind desert hurry useless lonely frozen morning', 'None') ;


In [44]:
res

True

In [46]:
res = new_address(db, "first_address", "m/0'/0'/", 0, 0, 111111111, xprv)
res

True

In [47]:
res = new_utxo(db, "first_address", 22222, "transaction_0", 0, 111111112, confirmed=6)
res

True

In [50]:
utxos = look_for_coins(db, xprv)

In [51]:
utxos

[('transaction_0', 0, 22222)]

In [56]:
unused_addresses = get_unused_addresses(db,xprv)
unused_addresses

[]

In [57]:
res = new_address(db, "second_address", "m/0'/0'/", 1, 1, 111111111, xprv)
res

True

In [65]:
unused_addresses = get_unused_addresses(db,xprv)
unused_addresses

SELECT address 
FROM Addresses WHERE
NOT EXISTS(
SELECT 1 
 FROM Utxos
WHERE Utxos.address = Addresses.address) 
AND
 Addresses.wallet = 'tprv8ZgxMBicQKsPfQJYjuFAso9x6STzmUdMh5U8CQqqQUTgtQHBHCq4C7FseeeZg15L16UeSwbrLwJRTXNPQsJQwqvbBA11sn4M6c3jR1LwAQP';


[('second_address',)]

In [59]:
f = db.cursor().execute("SELECT * FROM Addresses")

In [61]:
f.fetchall()

[('first_address',
  0,
  "m/0'/0'/",
  0,
  111111111,
  'tprv8ZgxMBicQKsPfQJYjuFAso9x6STzmUdMh5U8CQqqQUTgtQHBHCq4C7FseeeZg15L16UeSwbrLwJRTXNPQsJQwqvbBA11sn4M6c3jR1LwAQP'),
 ('second_address',
  1,
  "m/0'/0'/",
  1,
  111111111,
  'tprv8ZgxMBicQKsPfQJYjuFAso9x6STzmUdMh5U8CQqqQUTgtQHBHCq4C7FseeeZg15L16UeSwbrLwJRTXNPQsJQwqvbBA11sn4M6c3jR1LwAQP')]